In [1]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (NaN-SAFE)
# NaNs are strictly ignored (no interpolation, no filling)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATHS (USER PROVIDED)
# ============================================================

files = {
    'CO':     r"D:\IPMA\Results\co_fire_meteo_Iberia.nc",
    'NO':     r"D:\IPMA\Results\no_fire_meteo_Iberia.nc",
    'NO2':    r"D:\IPMA\Results\no2_fire_meteo_Iberia.nc",
    'PM2p5':  r"D:\IPMA\Results\pm2p5_fire_meteo_Iberia.nc",
    'PM10':   r"D:\IPMA\Results\pm10_fire_meteo_Iberia.nc"
}

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"
TEST_YEAR = 2017
TEST_MONTH = None        # None or 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

INPUT_VARS = [
    "Mean",
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# STORAGE
# ============================================================

all_predictions = {}
all_metrics = {}

# ============================================================
# MAIN LOOP
# ============================================================

for pollutant, path in files.items():

    print(f"\n==============================")
    print(f"Processing: {pollutant}")
    print(f"==============================")

    ds = xr.open_dataset(path)

    # -------------------------------
    # CHECK VARIABLES
    # -------------------------------
    for var in INPUT_VARS:
        if var not in ds:
            raise KeyError(f"{var} not found in {path}")

    y = ds[POLLUTANT_VAR]
    X = xr.merge([ds[var] for var in INPUT_VARS])

    time = ds.time
    train_idx = train_mask(time)
    test_idx = test_mask(time)

    X_train = X.sel(time=train_idx)
    y_train = y.sel(time=train_idx)
    X_test = X.sel(time=test_idx)
    y_test = y.sel(time=test_idx)

    # ========================================================
    # BUILD TRAINING DATA (NaN-SAFE)
    # ========================================================

    X_all, y_all = [], []

    for lat in ds.latitude.values:
        for lon in ds.longitude.values:

            X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
            y_ts = y_train.sel(latitude=lat, longitude=lon).values

            # Keep only valid days (no NaNs anywhere)
            valid = (
                ~np.isnan(y_ts) &
                ~np.isnan(X_ts).any(axis=1)
            )

            X_ts = X_ts[valid]
            y_ts = y_ts[valid]

            if len(y_ts) <= LAG_DAYS:
                continue

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()

            X_scaled = scaler_X.fit_transform(X_ts)
            y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

            X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

            if len(y_seq) == 0:
                continue

            X_all.append(X_seq)
            y_all.append(y_seq)

    X_train_all = np.concatenate(X_all)
    y_train_all = np.concatenate(y_all)

    print(f"Training samples: {X_train_all.shape[0]}")

    # ========================================================
    # MODEL
    # ========================================================

    model = Sequential([
        LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")

    model.fit(
        X_train_all,
        y_train_all,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    # ========================================================
    # PREDICTION (NaN-SAFE)
    # ========================================================

    test_times = time.sel(time=test_idx)
    preds = np.full(
        (len(ds.latitude), len(ds.longitude), len(test_times)),
        np.nan
    )

    for i, lat in enumerate(ds.latitude.values):
        for j, lon in enumerate(ds.longitude.values):

            X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
            y_ts = y.sel(latitude=lat, longitude=lon).values

            valid = ~np.isnan(X_ts).any(axis=1)

            if valid.sum() <= LAG_DAYS:
                continue

            X_ts = X_ts[valid]
            y_ts = y_ts[valid]

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()

            X_scaled = scaler_X.fit_transform(X_ts)
            y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

            test_start = np.where(test_idx.values)[0][0]

            for k in range(len(test_times)):
                t = test_start + k
                if t - LAG_DAYS < 0:
                    continue

                X_input = X_scaled[t-LAG_DAYS:t]

                if np.isnan(X_input).any():
                    continue

                pred_scaled = model.predict(
                    X_input.reshape(1, LAG_DAYS, -1),
                    verbose=0
                )

                preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

    # ========================================================
    # OUTPUT
    # ========================================================

    pred_da = xr.DataArray(
        preds,
        dims=("latitude", "longitude", "time"),
        coords={
            "latitude": ds.latitude,
            "longitude": ds.longitude,
            "time": test_times
        },
        name=f"{pollutant}_predicted"
    )

    all_predictions[pollutant] = pred_da

    # ========================================================
    # EVALUATION (NaN-SAFE)
    # ========================================================

    error = pred_da - y_test

    mae = float(abs(error).where(np.isfinite(error)).mean())
    rmse = float(np.sqrt((error ** 2)).where(np.isfinite(error)).mean())
    bias = float(error.where(np.isfinite(error)).mean())
    corr = float(
        xr.corr(pred_da, y_test, dim="time")
        .where(np.isfinite(pred_da))
        .mean()
    )

    all_metrics[pollutant] = {
        "MAE": mae,
        "RMSE": rmse,
        "Bias": bias,
        "Correlation": corr
    }

    print("Evaluation:")
    for k, v in all_metrics[pollutant].items():
        print(f"  {k}: {v:.4f}")

# ============================================================
# FINAL SUMMARY
# ============================================================

print("\n==============================")
print("FINAL SUMMARY (2017)")
print("==============================")

for pol, metrics in all_metrics.items():
    print(f"\n{pol}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")

# Optional save:
# xr.Dataset(all_predictions).to_netcdf("pollutant_predictions_2017.nc")


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):



Processing: CO
Training samples: 1139999


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 176s 5ms/step - loss: 0.0012
Epoch 2/40
   19/35625 ━━━━━━━━━━━━━━━━━━━━ 3:24 6ms/step - loss: 9.1016e-04

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


...

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 121s 3ms/step - loss: 0.0014
Epoch 2/40
   27/35625 ━━━━━━━━━━━━━━━━━━━━ 2:22 4ms/step - loss: 0.0010 

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0013
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0013
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0013
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0013
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0013
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 179s 5ms/step - loss: 0.0013
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 182s 5ms/step - loss: 0.0012
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 168s 5ms/step - loss: 0.0012
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0012
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0012
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0012
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0012
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0012
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/ste

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
# ============================================================
# DAILY POLLUTANT PREDICTION WITH LSTM (PER-GRID)
# Using explicit file paths (IPMA Iberia)
# ============================================================

import numpy as np
import xarray as xr

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# FILE PATHS (USER-PROVIDED)
# ============================================================

files = {
    'CO':     r"D:\IPMA\Results\co_fire_meteo_Iberia.nc",
    'NO':     r"D:\IPMA\Results\no_fire_meteo_Iberia.nc",
    'NO2':    r"D:\IPMA\Results\no2_fire_meteo_Iberia.nc",
    'PM2p5':  r"D:\IPMA\Results\pm2p5_fire_meteo_Iberia.nc",
    'PM10':   r"D:\IPMA\Results\pm10_fire_meteo_Iberia.nc"
}

# ============================================================
# USER SETTINGS
# ============================================================

POLLUTANT_VAR = "Mean"      # daily mean concentration
TEST_YEAR = 2017
TEST_MONTH = None           # None or integer 1–12
LAG_DAYS = 14
EPOCHS = 40
BATCH_SIZE = 32

# Input variables for the model
INPUT_VARS = [
    "Mean",                          # lagged pollutant
    "temp_Max",
    "wind_Max",
    "precip_Total_Precipitation",
    "frp_sum_Iberia"
]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def build_sequences(X, y, lags):
    X_out, y_out = [], []
    for i in range(lags, len(X)):
        X_out.append(X[i-lags:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)


def train_mask(time):
    return time.dt.year != TEST_YEAR


def test_mask(time):
    mask = time.dt.year == TEST_YEAR
    if TEST_MONTH is not None:
        mask = mask & (time.dt.month == TEST_MONTH)
    return mask

# ============================================================
# STORAGE
# ============================================================

all_predictions = {}
all_metrics = {}

# ============================================================
# MAIN LOOP OVER POLLUTANTS
# ============================================================

for pollutant, path in files.items():

    print(f"\n==============================")
    print(f"Processing pollutant: {pollutant}")
    print(f"File: {path}")
    print(f"==============================")

    ds = xr.open_dataset(path)

    # -------------------------------
    # CHECK REQUIRED VARIABLES
    # -------------------------------
    for var in INPUT_VARS:
        if var not in ds:
            raise KeyError(f"{var} not found in {path}")

    # -------------------------------
    # DEFINE INPUTS / TARGET
    # -------------------------------
    y = ds[POLLUTANT_VAR]
    X = xr.merge([ds[var] for var in INPUT_VARS])

    time = ds.time
    train_idx = train_mask(time)
    test_idx = test_mask(time)

    X_train = X.sel(time=train_idx)
    y_train = y.sel(time=train_idx)

    X_test = X.sel(time=test_idx)
    y_test = y.sel(time=test_idx)

    # -------------------------------
    # BUILD TRAINING DATASET
    # -------------------------------
    X_all, y_all = [], []

    for lat in ds.latitude.values:
        for lon in ds.longitude.values:

            X_ts = X_train.sel(latitude=lat, longitude=lon).to_array().values.T
            y_ts = y_train.sel(latitude=lat, longitude=lon).values

            if np.isnan(X_ts).any() or np.isnan(y_ts).any():
                continue

            if len(y_ts) <= LAG_DAYS:
                continue

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()

            X_scaled = scaler_X.fit_transform(X_ts)
            y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1)).ravel()

            X_seq, y_seq = build_sequences(X_scaled, y_scaled, LAG_DAYS)

            X_all.append(X_seq)
            y_all.append(y_seq)

    X_train_all = np.concatenate(X_all)
    y_train_all = np.concatenate(y_all)

    print(f"Training samples: {X_train_all.shape[0]}")

    # -------------------------------
    # LSTM MODEL
    # -------------------------------
    model = Sequential([
        LSTM(64, input_shape=(LAG_DAYS, X_train_all.shape[2])),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")

    model.fit(
        X_train_all,
        y_train_all,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    # -------------------------------
    # PREDICTION FOR TEST PERIOD
    # -------------------------------
    test_times = time.sel(time=test_idx)
    preds = np.zeros((len(ds.latitude), len(ds.longitude), len(test_times)))

    for i, lat in enumerate(ds.latitude.values):
        for j, lon in enumerate(ds.longitude.values):

            X_ts = X.sel(latitude=lat, longitude=lon).to_array().values.T
            y_ts = y.sel(latitude=lat, longitude=lon).values

            if np.isnan(X_ts).any() or np.isnan(y_ts).any():
                preds[i, j, :] = np.nan
                continue

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()

            X_scaled = scaler_X.fit_transform(X_ts)
            y_scaled = scaler_y.fit_transform(y_ts.reshape(-1, 1))

            test_start = np.where(test_idx.values)[0][0]

            for k in range(len(test_times)):
                t = test_start + k
                X_input = X_scaled[t-LAG_DAYS:t]
                pred_scaled = model.predict(
                    X_input.reshape(1, LAG_DAYS, -1),
                    verbose=0
                )
                preds[i, j, k] = scaler_y.inverse_transform(pred_scaled)[0, 0]

    # -------------------------------
    # XR OUTPUT
    # -------------------------------
    pred_da = xr.DataArray(
        preds,
        dims=("latitude", "longitude", "time"),
        coords={
            "latitude": ds.latitude,
            "longitude": ds.longitude,
            "time": test_times
        },
        name=f"{pollutant}_predicted"
    )

    all_predictions[pollutant] = pred_da

    # -------------------------------
    # EVALUATION METRICS
    # -------------------------------
    error = pred_da - y_test

    mae = float(abs(error).mean())
    rmse = float(np.sqrt((error ** 2).mean()))
    bias = float(error.mean())
    corr = float(xr.corr(pred_da, y_test, dim="time").mean())

    all_metrics[pollutant] = {
        "MAE": mae,
        "RMSE": rmse,
        "Bias": bias,
        "Correlation": corr
    }

    print("Evaluation metrics:")
    for k, v in all_metrics[pollutant].items():
        print(f"  {k}: {v:.4f}")

# ============================================================
# FINAL SUMMARY
# ============================================================

print("\n==============================")
print("FINAL SUMMARY (2017 prediction)")
print("==============================")

for pol, metrics in all_metrics.items():
    print(f"\n{pol}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")

# Optional save:
# xr.Dataset(all_predictions).to_netcdf("pollutant_predictions_2017.nc")



Processing pollutant: CO
File: D:\IPMA\Results\co_fire_meteo_Iberia.nc
Training samples: 1139999


C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 174s 5ms/step - loss: 0.0012
Epoch 2/40
   11/35625 ━━━━━━━━━━━━━━━━━━━━ 3:01 5ms/step - loss: 8.7911e-04

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


35625/35625 ━━━━━━━━━━━━━━━━━━━━ 176s 5ms/step - loss: 0.0011
Epoch 3/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 179s 5ms/step - loss: 0.0011
Epoch 4/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 158s 4ms/step - loss: 0.0011
Epoch 5/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 165s 3ms/step - loss: 0.0011
Epoch 6/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 129s 4ms/step - loss: 0.0011
Epoch 7/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0010
Epoch 8/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0010
Epoch 9/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0010
Epoch 10/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0010
Epoch 11/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 0.0010
Epoch 12/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 131s 4ms/step - loss: 0.0010  
Epoch 13/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 130s 4ms/step - loss: 0.0010  
Epoch 14/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 132s 4ms/step - loss: 9.9550e-04
Epoch 15/40
35625/35625 ━━━━━━━━━━━━━━━━━━━━ 153s

C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\sofia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Evaluation metrics:
  MAE: 0.0378
  RMSE: 0.4157
  Bias: 0.0070
  Correlation: 0.7204

Processing pollutant: NO
File: D:\IPMA\Results\no_fire_meteo_Iberia.nc
